In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as datetime
import matplotlib.dates as mdates
import edgar_utils as eu

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
ticker = 'FLS'
offset_fy = 0 # Need to figure of what the fy ends and assign 0 if fy matches calendar year or 1 otherwise

In [188]:
jj = eu.get_json_financials_from_tikr(ticker)

In [7]:
jj.tag.unique()

array(['AccountsPayableCurrent', 'AccountsReceivableGrossCurrent',
       'AccountsReceivableNetCurrent', 'AccountsReceivableNetNoncurrent',
       'AccruedEnvironmentalLossContingenciesNoncurrent',
       'AccruedIncomeTaxesCurrent', 'AccruedInsuranceNoncurrent',
       'AccruedLiabilitiesCurrent',
       'AccruedRoyaltiesCurrentAndNoncurrent',
       'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment',
       'AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax',
       'AccumulatedOtherComprehensiveIncomeLossDefinedBenefitPensionAndOtherPostretirementPlansNetOfTax',
       'AccumulatedOtherComprehensiveIncomeLossNetOfTax',
       'AdditionalPaidInCapitalCommonStock',
       'AdditionsToNoncurrentAssets', 'AdjustmentForAmortization',
       'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue',
       'AdjustmentsToAdditionalPaidInCapitalStockSplit',
       'Adjustmen

In [190]:
kk = jj[jj.form.isin(['10-K', '8-K'])]

In [167]:
bs_kk = kk[(kk.fy >= 2014)& (kk.end.dt.year == kk.fy)]

In [171]:
#kk[kk.tag.isin(['DepreciationDepletionAndAmortization', 'Depreciation'])]
bs_kk[(bs_kk.val==60434000)]
# bs_kk
# Found that this ticker doesn't have 'LongTermDebtCurrent' which is what the code looks for but has
# 'DebtCurrent' so will replace that before creating balance sheet

,end,val,accn,fy,fp,form,filed,frame,tag,units,start
4304,2015-12-31,60434000.0,0000030625-16-000121,2015,FY,10-K,2016-02-18,NaN,DebtCurrent,USD,NaN
11742,2015-12-31,60434000.0,0000030625-16-000121,2015,FY,10-K,2016-02-18,CY2015Q4I,LongTermDebtMaturitiesRepaymentsOfPrincipalInN...,USD,NaN


In [172]:
bs_kk.loc[bs_kk.tag=='DebtCurrent','tag']= 'LongTermDebtCurrent'

In [196]:
# Another investigation
bs_kk[bs_kk.val==228777000]

,end,val,accn,fy,fp,form,filed,frame,tag,units,start
10066,2015-12-31,228777000.0,0000030625-16-000121,2015,FY,10-K,2016-02-18,NaN,IntangibleAssetsNetExcludingGoodwill,USD,NaN


In [197]:
bs = eu.BalanceSheet(bs_kk,ticker, offset_fy, 2014)

WARN: Found AccountsPayableAndAccruedLiabilitiesCurrent; Converting to: AccountsPayable
WARN: Found AccountsPayableCurrent; Converting to: AccountsPayable
WARN: Found MarketableSecuritiesCurrent; Converting to: MarketableSecurities
WARN: Found IndefiniteLivedTradeNames; Converting to: IndefiniteLivedTrademarks
WARN: Found OtherIndefiniteLivedIntangibleAssets; Converting to: OtherIndefiniteLivedAndFiniteLivedIntangibleAssets
WARN: Found FiniteLivedIntangibleAssetsNet; Converting to: OtherIntangibleAssetsNet
WARN: Found IntangibleAssetsNetExcludingGoodwill; Converting to: OtherIntangibleAssetsNet


In [174]:
#bs.df.to_dict()
# bs.df.Assets - bs.df.StockholdersEquity
bs.df

tag,AccountsPayable,AccountsReceivableNetCurrent,AccruedIncomeTaxesCurrent,Assets,AssetsCurrent,CashAndCashEquivalentsAtCarryingValue,Goodwill,InventoryNet,Liabilities,LiabilitiesAndStockholdersEquity,LiabilitiesCurrent,LongTermDebtCurrent,LongTermDebtNoncurrent,MinorityInterest,OperatingLeaseLiabilityCurrent,RetainedEarningsAccumulatedDeficit,StockholdersEquity,TreasuryStockValue,OtherIntangibleAssetsNet,LiabilitiesOfDisposalGroupIncludingDiscontinuedOperationCurrent,CustomerRefundLiabilityCurrent,PreferredStockIncludingAdditionalPaidInCapitalNetOfDiscount,OtherIndefiniteLivedAndFiniteLivedIntangibleAssets,AccruedAdvertisingCurrent,ContractWithCustomerLiability,MarketableSecurities,IndefiniteLivedTrademarks,DerivativeLiabilitiesCurrent
fy,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014,611715000.0,1.082447e+09,37451000.0,4.968020e+09,2.794163e+09,4.503500e+08,1.067255e+09,995564000.0,3.036458e+09,4.968020e+09,1.471875e+09,53131000.0,1.101791e+09,10281000.0,NaN,3.415738e+09,1.931562e+09,1.830919e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,491378000.0,9.883910e+08,38747000.0,5.103850e+09,2.631792e+09,3.664440e+08,1.223986e+09,995565000.0,3.437373e+09,5.103850e+09,1.359962e+09,60434000.0,1.570836e+09,17256000.0,NaN,3.587120e+09,1.666477e+09,2.106785e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016,412087000.0,8.947490e+08,15755000.0,4.742762e+09,2.331361e+09,3.671620e+08,1.205054e+09,919251000.0,3.094528e+09,4.742762e+09,1.178141e+09,85365000.0,1.485258e+09,20961000.0,NaN,3.632163e+09,1.648234e+09,2.078527e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017,443113000.0,8.567110e+08,27862000.0,4.910474e+09,2.558745e+09,7.034450e+08,1.218188e+09,884273000.0,3.255887e+09,4.910474e+09,1.242908e+09,75599000.0,1.499658e+09,16367000.0,NaN,3.503947e+09,1.654587e+09,2.059558e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018,418893000.0,7.924340e+08,9865000.0,4.616277e+09,2.383145e+09,6.196830e+08,1.197640e+09,633871000.0,2.973962e+09,4.616277e+09,1.080975e+09,68218000.0,1.414829e+09,18465000.0,NaN,3.543007e+09,1.642315e+09,2.049404e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019,447582000.0,7.955380e+08,20018000.0,4.919642e+09,2.505370e+09,6.709800e+08,1.193010e+09,660837000.0,3.129285e+09,4.919642e+09,1.112888e+09,11272000.0,1.365977e+09,25602000.0,36108000.0,3.695862e+09,1.790357e+09,2.051583e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020,440199000.0,7.534620e+08,31378000.0,5.314677e+09,2.904333e+09,1.095274e+09,1.224886e+09,667228000.0,3.582207e+09,5.314677e+09,1.141633e+09,8995000.0,1.717911e+09,30330000.0,34990000.0,3.656449e+09,1.732470e+09,2.059309e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021,410062000.0,7.392100e+08,47186000.0,4.749768e+09,2.388677e+09,6.584520e+08,1.196479e+09,678287000.0,2.945449e+09,4.749768e+09,1.131805e+09,41058000.0,1.261770e+09,33026000.0,32628000.0,3.691023e+09,1.804319e+09,2.057706e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
bs.df.loc[:,['LongTermDebtNoncurrent', 'LongTermDebtCurrent']]

tag,LongTermDebtNoncurrent,LongTermDebtCurrent
fy,,
2014,1.101791e+09,53131000.0
2015,1.570836e+09,60434000.0
2016,1.485258e+09,85365000.0
2017,1.499658e+09,75599000.0
2018,1.414829e+09,68218000.0
2019,1.365977e+09,11272000.0
2020,1.717911e+09,8995000.0
2021,1.261770e+09,41058000.0


In [71]:
kk=jj[(jj.tag.str.contains("InventoryNet"))  & (jj.form.isin(['10-K','8-K'])) & (jj.end.dt.year == jj.fy)]
#jj[(jj.tag.str.contains("MarketableSecurities")) ]
#jj[jj.tag.isin(['NetCashProvidedByUsedInOperatingActivities','PaymentsToAcquirePropertyPlantAndEquipment'])]

In [59]:
kk = jj[(jj.tag.isin(["Revenues", "RevenueFromContractWithCustomerExcludingAssessedTax"]))]# & (jj.form.isin(['10-K','8-K'])) ]
# kk[(kk.fy=='2020') & (kk.form == '10-K') & (kk.frame == 'CY2020')]
kk[(kk.form == '10-K') & (kk.frame == 'CY2020')]

,end,val,accn,fy,fp,form,filed,frame,tag,units,start
18258,2020-12-31,3.728134e+09,0000030625-23-000077,2021,FY,10-K,2023-03-07,CY2020,RevenueFromContractWithCustomerExcludingAssess...,USD,2020-01-01
18332,2020-12-31,0.000000e+00,0000030625-23-000077,2021,FY,10-K,2023-03-07,CY2020,Revenues,USD,2020-01-01


In [73]:
jj[(jj.tag.isin(['OperatingIncomeLoss','NetIncomeLoss','EarningsPerShareDiluted', 'WeightedAverageNumberOfDilutedSharesOutstanding'])) & (jj.form=='10-K') & (jj.end.dt.year == jj.fy)]

,end,val,accn,fy,fp,form,filed,frame,tag,units,start
7446,2009-12-31,2.930000e+00,0001047469-10-001476,2009,FY,10-K,2010-02-26,NaN,EarningsPerShareDiluted,USD/shares,2009-01-01
7459,2010-12-31,5.060000e+00,0001047469-11-001506,2010,FY,10-K,2011-02-28,NaN,EarningsPerShareDiluted,USD/shares,2010-01-01
7473,2011-12-31,3.690000e+00,0000021344-12-000007,2011,FY,10-K,2012-02-23,NaN,EarningsPerShareDiluted,USD/shares,2011-01-01
7487,2012-12-31,1.970000e+00,0000021344-13-000007,2012,FY,10-K,2013-02-27,NaN,EarningsPerShareDiluted,USD/shares,2012-01-01
7501,2013-12-31,1.900000e+00,0000021344-14-000008,2013,FY,10-K,2014-02-27,NaN,EarningsPerShareDiluted,USD/shares,2013-01-01
7515,2014-12-31,1.600000e+00,0000021344-15-000005,2014,FY,10-K,2015-02-25,NaN,EarningsPerShareDiluted,USD/shares,2014-01-01
7529,2015-12-31,1.670000e+00,0000021344-16-000050,2015,FY,10-K,2016-02-25,NaN,EarningsPerShareDiluted,USD/shares,2015-01-01
7543,2016-12-31,1.490000e+00,0000021344-17-000009,2016,FY,10-K,2017-02-24,NaN,EarningsPerShareDiluted,USD/shares,2016-01-01
7557,2017-12-31,2.900000e-01,0000021344-18-000008,2017,FY,10-K,2018-02-23,NaN,EarningsPerShareDiluted,USD/shares,2017-01-01
7571,2018-12-31,1.500000e+00,0000021344-19-000014,2018,FY,10-K,2019-02-21,NaN,EarningsPerShareDiluted,USD/shares,2018-01-01


In [195]:
# In order to fix above, we'll have to verify by opening the 10Ks from the SEC and checking by hand
# In this case, I want 'Depreciation' to be used rather than 'DepreciationDepletionAndAmortization'

# income_kk = eu.filterPeriodStatement(kk)
# kk[kk.val == 789832000]
kk[kk.tag.isin(['OperatingIncomeLoss'])]# & (kk.fy==kk.end.dt.year)]
# income_kk

,end,val,accn,fy,fp,form,filed,frame,tag,units,start
13400,2007-12-31,411890000.0,0000950123-10-016384,2009,FY,10-K,2010-02-24,CY2007,OperatingIncomeLoss,USD,2007-01-01
13405,2008-12-31,615678000.0,0000950123-10-016384,2009,FY,10-K,2010-02-24,NaN,OperatingIncomeLoss,USD,2008-01-01
13406,2008-12-31,615678000.0,0000950123-11-017117,2010,FY,10-K,2011-02-23,CY2008,OperatingIncomeLoss,USD,2008-01-01
13416,2009-12-31,629517000.0,0000950123-10-016384,2009,FY,10-K,2010-02-24,NaN,OperatingIncomeLoss,USD,2009-01-01
13417,2009-12-31,629517000.0,0000950123-11-017117,2010,FY,10-K,2011-02-23,NaN,OperatingIncomeLoss,USD,2009-01-01
13418,2009-12-31,629517000.0,0000030625-12-000005,2011,FY,10-K,2012-02-22,NaN,OperatingIncomeLoss,USD,2009-01-01
13419,2009-12-31,629517000.0,0000030625-12-000018,2012,Q2,8-K,2012-08-30,CY2009,OperatingIncomeLoss,USD,2009-01-01
13430,2010-12-31,581352000.0,0000950123-11-017117,2010,FY,10-K,2011-02-23,NaN,OperatingIncomeLoss,USD,2010-01-01
13431,2010-12-31,581352000.0,0000030625-12-000005,2011,FY,10-K,2012-02-22,NaN,OperatingIncomeLoss,USD,2010-01-01
13432,2010-12-31,581352000.0,0000030625-12-000018,2012,Q2,8-K,2012-08-30,NaN,OperatingIncomeLoss,USD,2010-01-01


In [61]:
income = eu.IncomeStatement(jj,2014)

WARN: Found RevenueFromContractWithCustomerExcludingAssessedTax; Converting to: Revenues
WARN: Found GeneralAndAdministrativeExpense; Converting to: SellingGeneralAndAdministrativeExpense


In [66]:
income.df

tag,EarningsPerShareDiluted,GrossProfit,IncomeTaxExpenseBenefit,InterestExpense,NetIncomeLoss,OperatingIncomeLoss,Revenues,SellingGeneralAndAdministrativeExpense,WeightedAverageNumberOfDilutedSharesOutstanding,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,OperatingExpenses
fy,,,,,,,,,,,
2015,1.93,1.477537e+09,148351000.0,65270000.0,258411000.0,514665000.0,NaN,972733000.0,133811000.0,NaN,NaN
2016,1.01,1.236798e+09,77380000.0,60137000.0,132455000.0,276684000.0,3.990487e+09,965376000.0,130975000.0,NaN,NaN
2017,0.02,1.088953e+09,258679000.0,59730000.0,2652000.0,341135000.0,3.660831e+09,901727000.0,131358000.0,NaN,NaN
2018,0.80,1.190869e+09,46550000.0,58160000.0,104508000.0,227701000.0,3.835699e+09,966584000.0,131271000.0,NaN,NaN
2019,1.81,1.289343e+09,75493000.0,54980000.0,238828000.0,386623000.0,3.939697e+09,913203000.0,131719000.0,NaN,NaN
2020,1.00,1.116769e+09,61417000.0,56185000.0,130420000.0,250277000.0,3.728134e+09,878245000.0,131050000.0,NaN,NaN
2021,0.96,1.049725e+09,NaN,57617000.0,125949000.0,270759000.0,3.541060e+09,797076000.0,130857000.0,NaN,NaN
2022,1.44,9.942950e+08,NaN,46247000.0,188689000.0,197219000.0,3.615120e+09,815545000.0,131315000.0,NaN,NaN


In [75]:
# income.df
income.df.to_dict()

{'EarningsPerShareDiluted': {2014: 1.6,
  2015: 1.67,
  2016: 1.49,
  2017: 0.29,
  2018: 1.5,
  2019: 2.07,
  2020: 1.79,
  2021: 2.25,
  2022: 2.19},
 'GrossProfit': {2014: 28109000000.0,
  2015: 26812000000.0,
  2016: 25398000000.0,
  2017: 22491000000.0,
  2018: 21233000000.0,
  2019: 22647000000.0,
  2020: 19581000000.0,
  2021: 23298000000.0,
  2022: 25004000000.0},
 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest': {2014: 9325000000.0,
  2015: 9605000000.0,
  2016: 8136000000.0,
  2017: 6890000000.0,
  2018: 8225000000.0,
  2019: 10786000000.0,
  2020: 9749000000.0,
  2021: 12425000000.0,
  2022: 11686000000.0},
 'IncomeTaxExpenseBenefit': {2014: 2201000000.0,
  2015: 2239000000.0,
  2016: 1586000000.0,
  2017: 5607000000.0,
  2018: 1749000000.0,
  2019: 1801000000.0,
  2020: 1981000000.0,
  2021: 2621000000.0,
  2022: 2115000000.0},
 'InterestExpense': {2014: 483000000.0,
  2015: 856000000.0,
  2016: 733000000.0,
  2017: 853000000.0,

In [125]:
cashflow = eu.CashFlowStatement(jj,2014)

WARN: Found PaymentsToAcquireProductiveAssets; Converting to: PaymentsToAcquirePropertyPlantAndEquipment
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/madhu/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2354/2919515222.py", line 1, in <module>
    cashflow = eu.CashFlowStatement(jj,2014)
  File "/home/madhu/jupyter_notebooks/edgar_utils.py", line 222, in __init__
    raise ValueError('Both ' + k + ' and ' + v + ' found; Need to disambiguate')
ValueError: Both Depreciation and DepreciationDepletionAndAmortization found; Need to disambiguate

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/madhu/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/madhu/anaconda3/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structu

In [128]:
# In order to fix above, we'll have to verify by opening the 10Ks from the SEC and checking by hand
# In this case, I want 'Depreciation' to be used rather than 'DepreciationDepletionAndAmortization'

cf_kk = eu.filterPeriodStatement(kk)
cf_kk = cf_kk[cf_kk.tag.isin(['DepreciationDepletionAndAmortization', 'Depreciation'])]
cf_kk

# kk = jj[(jj.fy==2015) & (jj.form=='10-K')]
# # kk[(kk.end.dt.year==2015) & (kk.val==417092000)]
# kk = kk[~kk.frame.isna()]
# kk = kk[kk.frame.str.match('CY\d\d\d\d$')]
# kk[kk.frame=='CY2015']

,end,val,accn,fy,fp,form,filed,frame,tag,units,start
5455,2007-12-31,67836000.0,0000950123-10-016384,2007,FY,10-K,2010-02-24,CY2007,Depreciation,USD,2007-01-01
5459,2008-12-31,71584000.0,0000950123-11-017117,2008,FY,10-K,2011-02-23,CY2008,Depreciation,USD,2008-01-01
5468,2009-12-31,85585000.0,0000030625-12-000018,2009,Q2,8-K,2012-08-30,CY2009,Depreciation,USD,2009-01-01
5478,2010-12-31,90509000.0,0000030625-13-000019,2010,FY,10-K,2013-02-21,CY2010,Depreciation,USD,2010-01-01
5489,2011-12-31,90653000.0,0000030625-14-000017,2011,FY,10-K,2014-02-18,CY2011,Depreciation,USD,2011-01-01
5499,2012-12-31,88572000.0,0000030625-15-000021,2012,FY,10-K,2015-02-17,CY2012,Depreciation,USD,2012-01-01
5508,2013-12-31,90695000.0,0000030625-16-000121,2013,FY,10-K,2016-02-18,CY2013,Depreciation,USD,2013-01-01
5528,2015-12-31,99501000.0,0001628280-18-002486,2015,FY,10-K,2018-02-28,CY2015,Depreciation,USD,2015-01-01
5539,2016-12-31,99897000.0,0000030625-19-000009,2016,FY,10-K,2019-02-20,CY2016,Depreciation,USD,2016-01-01
5549,2017-12-31,101438000.0,0000030625-20-000007,2017,FY,10-K,2020-02-18,CY2017,Depreciation,USD,2017-01-01


In [131]:
# Once issue is identified, patch the dataframe:
cf_kk = eu.filterPeriodStatement(kk)
cf_kk = cf_kk[cf_kk.tag != 'DepreciationDepletionAndAmortization']
cf_kk.loc[cf_kk.tag=='Depreciation','tag']= 'DepreciationDepletionAndAmortization'

In [133]:
# Now retry
cashflow = eu.CashFlowStatement(cf_kk,2014)

WARN: Found PaymentsToAcquireProductiveAssets; Converting to: PaymentsToAcquirePropertyPlantAndEquipment
WARN: Found Depreciation; Converting to: DepreciationDepletionAndAmortization


In [134]:
cashflow.df

tag,DepreciationDepletionAndAmortization,NetCashProvidedByUsedInOperatingActivities,PaymentsForRepurchaseOfCommonStock,PaymentsToAcquirePropertyPlantAndEquipment,ShareBasedCompensation,ProceedsFromIssuanceOfCommonStock
fy,,,,,,
2015,99501000.0,440759000.0,303651000.0,181861000.0,34816000.0,NaN
2016,99897000.0,240476000.0,NaN,89699000.0,30213000.0,NaN
2017,101438000.0,311066000.0,NaN,61602000.0,22820000.0,NaN
2018,95820000.0,190831000.0,NaN,83993000.0,19912000.0,NaN
2019,92042000.0,324097000.0,15000000.0,75716000.0,23882000.0,NaN
2020,86175000.0,310537000.0,32112000.0,57405000.0,27252000.0,NaN
2021,85175000.0,250119000.0,17531000.0,54936000.0,29478000.0,NaN
2022,77636000.0,NaN,NaN,76287000.0,25530000.0,NaN


In [135]:
mizrahi = eu.Mizrahi(bs, income, cashflow)

In [136]:
mizrahi.report_qualitative()

,Sales,NPM,ROE,OperatingMargin,EPS,FCF,FCF Margin,CurrentRatio,Solvency (D/E)
fy,,,,,,,,,
2014,nan,nan,nan,nan,nan,nan,nan,Current Ratio : Good Level!,nan
2015,nan,nan,nan,nan,nan,nan,nan,Current Ratio : Good Level!,Solvency (D/E) YoY above 10% => Not good
2016,nan,nan,ROE YoY below 10%,nan,EPS YoY below 10%,FCF YoY below 10%,nan,Current Ratio : Good Level!,Solvency (D/E) YoY below 10% => Improving
2017,Sales YoY < 10%,NPM YoY under 10%,ROE YoY below 10%,Operating Margin YoY up 10%,EPS YoY below 10%,FCF YoY up 10%,FCF Margin YoY up 10%,Current Ratio : Good Level!,Solvency (D/E) YoY below 10% => Improving
2018,Sales YoY < 10%,NPM YoY up 10%,ROE YoY up 10%,Operating Margin YoY below 10%,EPS YoY up 10%,FCF YoY below 10%,FCF Margin YoY below 10%,Current Ratio : Good Level!,Solvency (D/E) YoY below 10% => Improving
2019,Sales YoY < 10%,NPM YoY up 10%,ROE YoY up 10%,Operating Margin YoY up 10%,EPS YoY up 10%,FCF YoY up 10%,FCF Margin YoY up 10%,Current Ratio : Good Level!,Solvency (D/E) YoY below 10% => Improving
2020,Sales YoY < 10%,NPM YoY under 10%,ROE YoY below 10%,Operating Margin YoY below 10%,EPS YoY below 10%,FCF YoY below 10%,FCF Margin YoY below 10%,Current Ratio : Good Level!,Solvency (D/E) YoY above 10% => Not good
2021,Sales YoY < 10%,NPM YoY under 10%,ROE YoY below 10%,Operating Margin YoY up 10%,EPS YoY below 10%,FCF YoY below 10%,FCF Margin YoY below 10%,Current Ratio : Good Level!,Solvency (D/E) YoY below 10% => Improving


In [137]:
mizrahi.report_quantitative()

,Sales,Sales_YoY,NPM,NPM_YoY,ROE,ROE_YoY,OperatingMargin,OperatingMargin_YoY,EPS-DILUTED,EPS_YoY,FCF,FCF_YoY,FCF_Margin,FCF_Margin_YoY,CurrentRatio,Solvency (D/E Ratio),Solvency_YoY
fy,,,,,,,,,,,,,,,,,
2014,nan,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan,nan%,nan,nan%,nan%,nan%,1.898370,0.570415,nan
2015,nan,nan%,nan%,nan%,15.51%,nan%,nan%,nan%,1.930000,nan%,"258,898,000",nan%,nan%,nan%,1.935195,0.942609,0.652498
2016,"3,990,487,000",nan%,3.32%,nan%,8.04%,-48.18%,6.93%,nan%,1.010000,-47.67%,"150,777,000",-41.76%,3.78%,nan%,1.978847,0.901121,-0.044014
2017,"3,660,831,000",-8.26%,0.07%,-97.82%,0.16%,-98.01%,9.32%,34.40%,0.020000,-98.02%,"249,464,000",65.45%,6.81%,80.35%,2.058676,0.906364,0.005818
2018,"3,835,699,000",4.78%,2.72%,"3,661.07%",6.36%,"3,870.17%",5.94%,-36.29%,0.800000,"3,900.00%","106,838,000",-57.17%,2.79%,-59.13%,2.204625,0.861485,-0.049516
2019,"3,939,697,000",2.71%,6.06%,122.49%,13.34%,109.63%,9.81%,65.31%,1.810000,126.25%,"248,381,000",132.48%,6.30%,126.35%,2.251233,0.762963,-0.114362
2020,"3,728,134,000",-5.37%,3.50%,-42.29%,7.53%,-43.57%,6.71%,-31.59%,1.000000,-44.75%,"253,132,000",1.91%,6.79%,7.70%,2.544016,0.991596,0.299664
2021,"3,541,060,000",-5.02%,3.56%,1.67%,6.98%,-7.27%,7.65%,13.90%,0.960000,-4.00%,"195,183,000",-22.89%,5.51%,-18.82%,2.110502,0.699305,-0.294768


In [138]:
safal = eu.Safal(bs, income, cashflow)

In [139]:
safal.get_market_metrics('FLS')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/madhu/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2354/3505173965.py", line 1, in <module>
    safal.get_market_metrics('FLS')
  File "/home/madhu/jupyter_notebooks/edgar_utils.py", line 427, in get_market_metrics
    ticker_info = yf.Ticker(ticker).info
  File "/home/madhu/anaconda3/lib/python3.10/site-packages/yfinance/ticker.py", line 138, in info
    return self.get_info()
  File "/home/madhu/anaconda3/lib/python3.10/site-packages/yfinance/base.py", line 1102, in get_info
    data = self._quote.info
  File "/home/madhu/anaconda3/lib/python3.10/site-packages/yfinance/scrapers/quote.py", line 572, in info
    self._fetch(self.proxy)
  File "/home/madhu/anaconda3/lib/python3.10/site-packages/yfinance/scrapers/quote.py", line 723, in _fetch
    result = self._data.get_raw_json(
  File "/home/madhu/anaconda3/li

In [140]:
safal.report_qualitative()

AttributeError: 'Safal' object has no attribute 'pe'

In [141]:
safal.report_quantitative()

,GrossProfit,GrossProfit_YoY,Gross Margin,Gross Margin_YoY,ROE,ROE_YoY
fy,,,,,,
2014,nan,nan%,nan%,nan%,nan%,nan%
2015,"1,477,537,000",nan%,nan%,nan%,15.51%,nan%
2016,"1,236,798,000",-16.29%,30.99%,nan%,8.04%,-48.18%
2017,"1,088,953,000",-11.95%,29.75%,-4.03%,0.16%,-98.01%
2018,"1,190,869,000",9.36%,31.05%,4.37%,6.36%,"3,870.17%"
2019,"1,289,343,000",8.27%,32.73%,5.41%,13.34%,109.63%
2020,"1,116,769,000",-13.38%,29.96%,-8.47%,7.53%,-43.57%
2021,"1,049,725,000",-6.00%,29.64%,-1.04%,6.98%,-7.27%


In [142]:
threebrians = eu.ThreeBrians(bs, income, cashflow)

In [143]:
threebrians.report_qualitative()

,QuickRatio,CurrentRatio,Solvency (D/E),Goodwill-to-Assets,LessCashThanDebt?,IntangiblesTooHigh?,Goodwill Writedowns?,GrossProfit Growth,OperatingMargin Growth,NPM Growth,EPS Growth,SharesBoughtBack?,ExpenseGrowth,SGASlowdown?,SalesGrowth>ExpenseGrowth,OperatingLeverage?,OperatingCashFlow,CapEx,FCF,SBC,Deprec>CapEx?
fy,,,,,,,,,,,,,,,,,,,,,
2014,nan,CurrentRatio: Robust,Solvency is Anti-fragile,Goodwill-to-Assets: b/w .1 & .5 => Robust,Less Cash than Debt: YELLOW FLAG,Intangibles < 50% of assets: OK,nan,nan,nan,nan,nan,nan,nan,Not sure what to make of it,nan,nan,nan,nan,nan,nan,nan
2015,nan,CurrentRatio: Robust,Solvency is Anti-fragile,Goodwill-to-Assets: b/w .1 & .5 => Robust,Less Cash than Debt: YELLOW FLAG,Intangibles < 50% of assets: OK,Goodwill steady or increasing: Cool,nan,nan,nan,nan,nan,nan,Not sure what to make of it,nan,nan,nan,nan,nan,nan,Depreciation < CapEx; OK
2016,nan,CurrentRatio: Robust,Solvency is Anti-fragile,Goodwill-to-Assets: b/w .1 & .5 => Robust,Less Cash than Debt: YELLOW FLAG,Intangibles < 50% of assets: OK,Goodwill writedowns: Yellow Flag,Gross Profit YoY flat/shrinking,nan,nan,EPS YoY is flat/shrinking,Shares Being Bought Back!,nan,SGA as % of gross profit going up :(,nan,nan,Operating Cash Flow YoY is greater than NetInc...,Capex investment going down :(,FCF YoY going down :(,SBC seems OK,Depreciation > CapEx; Yellow Flag :(
2017,nan,CurrentRatio: Robust,Solvency is Anti-fragile,Goodwill-to-Assets: b/w .1 & .5 => Robust,Less Cash than Debt: YELLOW FLAG,Intangibles < 50% of assets: OK,Goodwill steady or increasing: Cool,Gross Profit YoY flat/shrinking,Oper Margin YoY is growing,NPM YoY is flat/shrinking,EPS YoY is flat/shrinking,Net Shares buyback not happening,nan,SGA as % of gross profit going up :(,nan,nan,Operating Cash Flow YoY is greater than NetInc...,Capex investment going down :(,FCF YoY going up :),SBC seems OK,Depreciation > CapEx; Yellow Flag :(
2018,nan,CurrentRatio: Robust,Solvency is Anti-fragile,Goodwill-to-Assets: b/w .1 & .5 => Robust,Less Cash than Debt: YELLOW FLAG,Intangibles < 50% of assets: OK,Goodwill writedowns: Yellow Flag,Gross Profit YoY is growing,Oper Margin YoY is flat/shrinking,NPM YoY is growing,EPS YoY is growing,Shares Being Bought Back!,nan,SGA as % of gross profit going down :),nan,nan,Operating Cash Flow YoY is less than NetIncome...,Capex investment going up :),FCF YoY going down :(,SBC seems OK,Depreciation > CapEx; Yellow Flag :(
2019,nan,CurrentRatio: Robust,Solvency is Anti-fragile,Goodwill-to-Assets: b/w .1 & .5 => Robust,Less Cash than Debt: YELLOW FLAG,Intangibles < 50% of assets: OK,Goodwill writedowns: Yellow Flag,Gross Profit YoY is growing,Oper Margin YoY is growing,NPM YoY is growing,EPS YoY is growing,Net Shares buyback not happening,nan,SGA as % of gross profit going down :),nan,All margins growing faster than revenues: Oper...,Operating Cash Flow YoY is less than NetIncome...,Capex investment going down :(,FCF YoY going up :),SBC seems OK,Depreciation > CapEx; Yellow Flag :(
2020,nan,CurrentRatio: Antifragile,Solvency is Anti-fragile,Goodwill-to-Assets: b/w .1 & .5 => Robust,Less Cash than Debt: YELLOW FLAG,Intangibles < 50% of assets: OK,Goodwill steady or increasing: Cool,Gross Profit YoY flat/shrinking,Oper Margin YoY is flat/shrinking,NPM YoY is flat/shrinking,EPS YoY is flat/shrinking,Shares Being Bought Back!,nan,SGA as % of gross profit going up :(,nan,nan,Operating Cash Flow YoY is greater than NetInc...,Capex investment going down :(,FCF YoY going up :),SBC seems OK,Depreciation > CapEx; Yellow Flag :(
2021,nan,CurrentRatio: Robust,Solvency is Anti-fragile,Goodwill-to-Assets: b/w .1 & .5 => Robust,Less Cash than Debt: YELLOW FLAG,Intangibles < 50% of assets: OK,Goodwill writedowns: Yellow Flag,Gross Profit YoY flat/shrinking,Oper Margin YoY is growing,NPM YoY is growing,EPS YoY is flat/shrinking,Shares Being Bought Back!,nan,SGA as % of gross profit going down :),nan,nan,Operating Cash Flow YoY is less than NetIncome...,Capex investment 

In [144]:
pd.set_option('display.max_columns', None)

In [145]:
threebrians.report_quantitative()

,QuickRatio,CurrentRatio,Solvency (D/E Ratio),Solvency_YoY,Goodwill-to-Assets,GtoA_YoY,Cash,Intangibles,Goodwill_YoY,Sales,Sales_YoY,GrossProfit,GrossProfit_YoY,Gross Margin,Gross Margin_YoY,OperatingMargin,OperatingMargin_YoY,NPM,NPM_YoY,EPS-DILUTED,EPS_YoY,No. Shares Diluted,SharesOutstanding_YoY,OperatingExpenses_YoY,SGA%,SGA%_YoY,OperExpenses,OperExpenses_YoY,OperatingCashFlow,OperCashFlow_YoY,NetIncome,NetIncome_YoY,CapEx,FCF,CapEx_YoY,FCF_YoY,FCF Margin,FCF Margin_YoY,SBC%,SBC_YoY,Depreciation,Equity,ROE,ROE_YoY
fy,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014,NaN,1.898370,0.570415,NaN,0.214825,NaN,4.503500e+08,1.067255e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.931562e+09,NaN,NaN
2015,NaN,1.935195,0.942609,0.652498,0.239816,0.116333,3.664440e+08,1.223986e+09,0.146854,NaN,NaN,1.477537e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.93,NaN,133811000.0,NaN,NaN,0.658348,NaN,NaN,NaN,440759000.0,NaN,258411000.0,NaN,181861000.0,258898000.0,NaN,NaN,NaN,NaN,NaN,NaN,99501000.0,1.666477e+09,0.155064,NaN
2016,NaN,1.978847,0.901121,-0.044014,0.254083,0.059489,3.671620e+08,1.205054e+09,-0.015467,3.990487e+09,NaN,1.236798e+09,-0.162933,0.309937,NaN,0.069336,NaN,0.033193,NaN,1.01,-0.476684,130975000.0,-0.021194,NaN,0.780545,0.185612,NaN,NaN,240476000.0,-0.454405,132455000.0,-0.487425,89699000.0,150777000.0,-0.506772,-0.417620,0.037784,NaN,0.007571,NaN,99897000.0,1.648234e+09,0.080362,-0.481752
2017,NaN,2.058676,0.906364,0.005818,0.248080,-0.023627,7.034450e+08,1.218188e+09,0.010899,3.660831e+09,-0.082610,1.088953e+09,-0.119539,0.297461,-0.040253,0.093185,0.343967,0.000724,-0.978175,0.02,-0.980198,131358000.0,0.002924,NaN,0.828068,0.060885,NaN,NaN,311066000.0,0.293543,2652000.0,-0.979978,61602000.0,249464000.0,-0.313236,0.654523,0.068144,0.803512,0.006234,-0.176681,101438000.0,1.654587e+09,0.001603,-0.980055
2018,NaN,2.204625,0.861485,-0.049516,0.259439,0.045788,6.196830e+08,1.197640e+09,-0.016868,3.835699e+09,0.047767,1.190869e+09,0.093591,0.310470,0.043734,0.059364,-0.362950,0.027246,36.610679,0.80,39.000000,131271000.0,-0.000662,NaN,0.811663,-0.019811,NaN,NaN,190831000.0,-0.386526,104508000.0,38.407240,83993000.0,106838000.0,0.363478,-0.571730,0.027854,-0.591254,0.005191,-0.167212,95820000.0,1.642315e+09,0.063635,38.701706
2019,NaN,2.251233,0.762963,-0.114362,0.242499,-0.065292,6.709800e+08,1.193010e+09,-0.003866,3.939697e+09,0.027113,1.289343e+09,0.082691,0.327270,0.054111,0.098135,0.653120,0.060621,1.224935,1.81,1.262500,131719000.0,0.003413,NaN,0.708270,-0.127384,NaN,NaN,324097000.0,0.698346,238828000.0,1.285260,75716000.0,248381000.0,-0.098544,1.324838,0.063046,1.263468,0.006062,0.167717,92042000.0,1.790357e+09,0.133397,1.096296
2020,NaN,2.544016,0.991596,0.299664,0.230472,-0.049596,1.095274e+09,1.224886e+09,0.026719,3.728134e+09,-0.053700,1.116769e+09,-0.133846,0.299552,-0.084694,0.067132,-0.315924,0.034983,-0.422928,1.00,-0.447514,131050000.0,-0.005079,NaN,0.786416,0.110334,NaN,NaN,310537000.0,-0.041839,130420000.0,-0.453917,57405000.0,253132000.0,-0.241838,0.019128,0.067898,0.076961,0.007310,0.205866,86175000.0,1.732470e+09,0.075280,-0.435670
2021,NaN,2.110502,0.699305,-0.294768,0.251903,0.092984,6.584520e+08,1.196479e+09,-0.023192,3.541060e+09,-0.050179,1.049725e+09,-0.060034,0.296444,-0.010376,0.076463,0.138991,0.035568,0.016737,0.96,-0.040000,130857000.0,-0.001473,NaN,0.759319,-0.034456,NaN,NaN,250119000.0,-0.194560,125949000.0,-0.034282,54936000.0,195183000.0,-0.043010,-0.228928,0.055120,-0.188192,0.008325,0.138827,85175000.0,1.804319e+09,0.069804,-0.072737


In [146]:
threebrians.metrics

,QuickRatio,CurrentRatio,Solvency (D/E Ratio),Solvency_YoY,Goodwill-to-Assets,GtoA_YoY,Cash,Intangibles,Goodwill_YoY,Sales,Sales_YoY,GrossProfit,GrossProfit_YoY,Gross Margin,Gross Margin_YoY,OperatingMargin,OperatingMargin_YoY,NPM,NPM_YoY,EPS-DILUTED,EPS_YoY,No. Shares Diluted,SharesOutstanding_YoY,OperatingExpenses_YoY,SGA%,SGA%_YoY,OperExpenses,OperExpenses_YoY,OperatingCashFlow,OperCashFlow_YoY,NetIncome,NetIncome_YoY,CapEx,FCF,CapEx_YoY,FCF_YoY,FCF Margin,FCF Margin_YoY,SBC%,SBC_YoY,Depreciation,Equity,ROE,ROE_YoY
fy,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014,NaN,1.898370,0.570415,NaN,0.214825,NaN,4.503500e+08,1.067255e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.931562e+09,NaN,NaN
2015,NaN,1.935195,0.942609,0.652498,0.239816,0.116333,3.664440e+08,1.223986e+09,0.146854,NaN,NaN,1.477537e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.93,NaN,133811000.0,NaN,NaN,0.658348,NaN,NaN,NaN,440759000.0,NaN,258411000.0,NaN,181861000.0,258898000.0,NaN,NaN,NaN,NaN,NaN,NaN,99501000.0,1.666477e+09,0.155064,NaN
2016,NaN,1.978847,0.901121,-0.044014,0.254083,0.059489,3.671620e+08,1.205054e+09,-0.015467,3.990487e+09,NaN,1.236798e+09,-0.162933,0.309937,NaN,0.069336,NaN,0.033193,NaN,1.01,-0.476684,130975000.0,-0.021194,NaN,0.780545,0.185612,NaN,NaN,240476000.0,-0.454405,132455000.0,-0.487425,89699000.0,150777000.0,-0.506772,-0.417620,0.037784,NaN,0.007571,NaN,99897000.0,1.648234e+09,0.080362,-0.481752
2017,NaN,2.058676,0.906364,0.005818,0.248080,-0.023627,7.034450e+08,1.218188e+09,0.010899,3.660831e+09,-0.082610,1.088953e+09,-0.119539,0.297461,-0.040253,0.093185,0.343967,0.000724,-0.978175,0.02,-0.980198,131358000.0,0.002924,NaN,0.828068,0.060885,NaN,NaN,311066000.0,0.293543,2652000.0,-0.979978,61602000.0,249464000.0,-0.313236,0.654523,0.068144,0.803512,0.006234,-0.176681,101438000.0,1.654587e+09,0.001603,-0.980055
2018,NaN,2.204625,0.861485,-0.049516,0.259439,0.045788,6.196830e+08,1.197640e+09,-0.016868,3.835699e+09,0.047767,1.190869e+09,0.093591,0.310470,0.043734,0.059364,-0.362950,0.027246,36.610679,0.80,39.000000,131271000.0,-0.000662,NaN,0.811663,-0.019811,NaN,NaN,190831000.0,-0.386526,104508000.0,38.407240,83993000.0,106838000.0,0.363478,-0.571730,0.027854,-0.591254,0.005191,-0.167212,95820000.0,1.642315e+09,0.063635,38.701706
2019,NaN,2.251233,0.762963,-0.114362,0.242499,-0.065292,6.709800e+08,1.193010e+09,-0.003866,3.939697e+09,0.027113,1.289343e+09,0.082691,0.327270,0.054111,0.098135,0.653120,0.060621,1.224935,1.81,1.262500,131719000.0,0.003413,NaN,0.708270,-0.127384,NaN,NaN,324097000.0,0.698346,238828000.0,1.285260,75716000.0,248381000.0,-0.098544,1.324838,0.063046,1.263468,0.006062,0.167717,92042000.0,1.790357e+09,0.133397,1.096296
2020,NaN,2.544016,0.991596,0.299664,0.230472,-0.049596,1.095274e+09,1.224886e+09,0.026719,3.728134e+09,-0.053700,1.116769e+09,-0.133846,0.299552,-0.084694,0.067132,-0.315924,0.034983,-0.422928,1.00,-0.447514,131050000.0,-0.005079,NaN,0.786416,0.110334,NaN,NaN,310537000.0,-0.041839,130420000.0,-0.453917,57405000.0,253132000.0,-0.241838,0.019128,0.067898,0.076961,0.007310,0.205866,86175000.0,1.732470e+09,0.075280,-0.435670
2021,NaN,2.110502,0.699305,-0.294768,0.251903,0.092984,6.584520e+08,1.196479e+09,-0.023192,3.541060e+09,-0.050179,1.049725e+09,-0.060034,0.296444,-0.010376,0.076463,0.138991,0.035568,0.016737,0.96,-0.040000,130857000.0,-0.001473,NaN,0.759319,-0.034456,NaN,NaN,250119000.0,-0.194560,125949000.0,-0.034282,54936000.0,195183000.0,-0.043010,-0.228928,0.055120,-0.188192,0.008325,0.138827,85175000.0,1.804319e+09,0.069804,-0.072737


In [147]:
buffett = eu.Buffett(bs, income, cashflow)

In [148]:
buffett.report_qualitative()

,GrossMargin,R&D,SGA,InterestExpense,DepreciationAmortizationExpense,IncomeTaxScrutiny,Asset Sale - Entry Other,Inventory,NetReceivables as Pct of Revenue,AdjDebtToEquity,RetainedEarnings,CapEx/NetIncome,NetSharesBuyBack
fy,,,,,,,,,,,,,
2014,nan,Too Much R&D? tbd,nan,nan,nan,nan,Asset Sale: Look for Entry other? TBD,Is Inventory going up alongside Net Income?,Is Receivables/Sales trending down? This indic...,Tsy-Adjusted D/E < .8 => Indicative of DCA :),Is Retained Earnings going up?,nan,nan
2015,nan,Too Much R&D? tbd,SGA b/w .3 & .8; Sometimes necessary to keep D...,Interest expense as % of OI: >= .1 => Company ...,Higher depreciation % of Gross Profits >= .1 =...,nan,Asset Sale: Look for Entry other? TBD,Is Inventory going up alongside Net Income?,Is Receivables/Sales trending down? This indic...,Tsy-Adjusted D/E < .8 => Indicative of DCA :),Is Retained Earnings going up?,CapEx/NetIncome > .5 => Expensive to maintain;...,nan
2016,Gross Margin implies tight competition,Too Much R&D? tbd,SGA b/w .3 & .8; Sometimes necessary to keep D...,Interest expense as % of OI: >= .1 => Company ...,Higher depreciation % of Gross Profits >= .1 =...,nan,Asset Sale: Look for Entry other? TBD,Is Inventory going up alongside Net Income?,Is Receivables/Sales trending down? This indic...,Tsy-Adjusted D/E < .8 => Indicative of DCA :),Is Retained Earnings going up?,CapEx/NetIncome > .5 => Expensive to maintain;...,nan
2017,Gross Margin implies tight competition,Too Much R&D? tbd,SGA > .8; Too excessive :(,Interest expense as % of OI: >= .1 => Company ...,Higher depreciation % of Gross Profits >= .1 =...,nan,Asset Sale: Look for Entry other? TBD,Is Inventory going up alongside Net Income?,Is Receivables/Sales trending down? This indic...,Tsy-Adjusted D/E < .8 => Indicative of DCA :),Is Retained Earnings going up?,CapEx/NetIncome > .5 => Expensive to maintain;...,nan
2018,Gross Margin implies tight competition,Too Much R&D? tbd,SGA > .8; Too excessive :(,Interest expense as % of OI: >= .1 => Company ...,Higher depreciation % of Gross Profits >= .1 =...,nan,Asset Sale: Look for Entry other? TBD,Is Inventory going up alongside Net Income?,Is Receivables/Sales trending down? This indic...,Tsy-Adjusted D/E < .8 => Indicative of DCA :),Is Retained Earnings going up?,CapEx/NetIncome > .5 => Expensive to maintain;...,nan
2019,Gross Margin implies tight competition,Too Much R&D? tbd,SGA b/w .3 & .8; Sometimes necessary to keep D...,Interest expense as % of OI: >= .1 => Company ...,Higher depreciation % of Gross Profits >= .1 =...,nan,Asset Sale: Look for Entry other? TBD,Is Inventory going up alongside Net Income?,Is Receivables/Sales trending down? This indic...,Tsy-Adjusted D/E < .8 => Indicative of DCA :),Is Retained Earnings going up?,CapEx/NetIncome b/w .25 & .5 => May have DCA :/,nan
2020,Gross Margin implies tight competition,Too Much R&D? tbd,SGA b/w .3 & .8; Sometimes necessary to keep D...,Interest expense as % of OI: >= .1 => Company ...,Higher depreciation % of Gross Profits >= .1 =...,nan,Asset Sale: Look for Entry other? TBD,Is Inventory going up alongside Net Income?,Is Receivables/Sales trending down? This indic...,Tsy-Adjusted D/E < .8 => Indicative of DCA :),Is Retained Earnings going up?,CapEx/NetIncome b/w .25 & .5 => May have DCA :/,nan
2021,Gross Margin implies tight competition,Too Much R&D? tbd,SGA b/w .3 & .8; Sometimes necessary to keep D...,Interest expense as % of OI: >= .1 => Company ...,Higher depreciation % of Gross Profits >= .1 =...,nan,Asset Sale: Look for Entry other? TBD,Is Inventory going up alongside Net Income?,Is Receivables/Sales trending down? This indic...,Tsy-Adjusted D/E < .8 => Indicative of DCA :),Is Retained Earnings going up?,CapEx/NetIncome b/w .25 & .5 => May have DCA :/,nan


In [149]:
buffett.report_quantitative()

,GrossMargin,R&D,NPM,SGA,InterestExpense,DepreciationAmortizationExpense,IncomeTaxManualCalc,ReportedTax,EPS-DILUTED,EPS_YoY,EBT,Inventory,NetIncome,NetReceivables as % of Sales,ROA,YearsofNItoPayLTD,AdjDebtToEquityRatio,RetainedEarnings,RetainedYoY,CapEx/NetIncome,NetSharesBuyback
fy,,,,,,,,,,,,,,,,,,,,,
2014,nan%,tbd,nan%,nan%,nan%,nan%,nan,nan,nan,nan,nan,"995,564,000",nan,nan,nan%,nan,29.28%,"3,415,738,000",nan%,nan%,nan
2015,nan%,tbd,nan%,65.83%,12.68%,19.33%,nan,"148,351,000",1.930000,nan,nan,"995,565,000",258411000.000000,nan,5.06%,7.000000,41.63%,"3,587,120,000",5.02%,70.38%,nan
2016,30.99%,tbd,3.32%,78.05%,21.73%,36.11%,nan,"77,380,000",1.010000,-0.476684,nan,"919,251,000",132455000.000000,0.224221,2.79%,12.000000,39.85%,"3,632,163,000",1.26%,67.72%,nan
2017,29.75%,tbd,0.07%,82.81%,17.51%,29.74%,nan,"258,679,000",0.020000,-0.980198,nan,"884,273,000",2652000.000000,0.234021,0.05%,566.000000,40.38%,"3,503,947,000",-3.53%,"2,322.85%",nan
2018,31.05%,tbd,2.72%,81.17%,25.54%,42.08%,nan,"46,550,000",0.800000,39.000000,nan,"633,871,000",104508000.000000,0.206594,2.26%,14.000000,38.32%,"3,543,007,000",1.11%,80.37%,nan
2019,32.73%,tbd,6.06%,70.83%,14.22%,23.81%,nan,"75,493,000",1.810000,1.262500,nan,"660,837,000",238828000.000000,0.201929,4.85%,6.000000,35.55%,"3,695,862,000",4.31%,31.70%,nan
2020,29.96%,tbd,3.50%,78.64%,22.45%,34.43%,nan,"61,417,000",1.000000,-0.447514,nan,"667,228,000",130420000.000000,0.202102,2.45%,14.000000,45.31%,"3,656,449,000",-1.07%,44.02%,nan
2021,29.64%,tbd,3.56%,75.93%,21.28%,31.46%,nan,nan,0.960000,-0.040000,nan,"678,287,000",125949000.000000,0.208754,2.65%,11.000000,32.67%,"3,691,023,000",0.95%,43.62%,nan


In [198]:
kjmarshall = eu.KJMarshall(ticker,bs, income, cashflow)

In [199]:
kjmarshall.report_qualitative()

,CapitalEmployed,Levered FCF,Unlevered FCF
fy,,,
2014,Check value history of Goodwill to see if need...,Assuming CapEx is all maintenance CapEx unless...,Find out tax rate and apply to Interest; then ...
2015,Check value history of Goodwill to see if need...,Assuming CapEx is all maintenance CapEx unless...,Find out tax rate and apply to Interest; then ...
2016,Check value history of Goodwill to see if need...,Assuming CapEx is all maintenance CapEx unless...,Find out tax rate and apply to Interest; then ...
2017,Check value history of Goodwill to see if need...,Assuming CapEx is all maintenance CapEx unless...,Find out tax rate and apply to Interest; then ...
2018,Check value history of Goodwill to see if need...,Assuming CapEx is all maintenance CapEx unless...,Find out tax rate and apply to Interest; then ...
2019,Check value history of Goodwill to see if need...,Assuming CapEx is all maintenance CapEx unless...,Find out tax rate and apply to Interest; then ...
2020,Check value history of Goodwill to see if need...,Assuming CapEx is all maintenance CapEx unless...,Find out tax rate and apply to Interest; then ...
2021,Check value history of Goodwill to see if need...,Assuming CapEx is all maintenance CapEx unless...,Find out tax rate and apply to Interest; then ...


In [200]:
kjmarshall.report_quantitative()

tag,Assets,CashAndCashEquivalentsAtCarryingValue,MarketableSecurities,AccountsPayable,AccruedIncomeTaxesCurrent,OperatingLeaseLiabilityCurrent,ContractWithCustomerLiability,AccruedAdvertisingCurrent,DerivativeLiabilitiesCurrent,LiabilitiesOfDisposalGroupIncludingDiscontinuedOperationCurrent,OperatingIncome,Levered FCF,InterestExpense,ShareholderEquity,No. Shares Diluted,Goodwill,IndefiniteLivedTrademarks,OtherIndefiniteLivedAndFiniteLivedIntangibleAssets,OtherIntangibleAssetsNet,Liabilities,TotalDebt,MinorityInterest,PreferredEquity
fy,,,,,,,,,,,,,,,,,,,,,,,
2014,4.968020e+09,4.503500e+08,NaN,611715000.0,37451000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.931562e+09,NaN,1.067255e+09,NaN,NaN,146337000.0,3.036458e+09,1.154922e+09,10281000.0,NaN
2015,5.103850e+09,3.664440e+08,NaN,491378000.0,38747000.0,NaN,NaN,NaN,NaN,NaN,514665000.0,258898000.0,65270000.0,1.666477e+09,133811000.0,1.223986e+09,NaN,NaN,228777000.0,3.437373e+09,1.631270e+09,17256000.0,NaN
2016,4.742762e+09,3.671620e+08,NaN,412087000.0,15755000.0,NaN,NaN,NaN,NaN,NaN,276684000.0,150777000.0,60137000.0,1.648234e+09,130975000.0,1.205054e+09,NaN,NaN,214527000.0,3.094528e+09,1.570623e+09,20961000.0,NaN
2017,4.910474e+09,7.034450e+08,NaN,443113000.0,27862000.0,NaN,NaN,NaN,NaN,NaN,341135000.0,249464000.0,59730000.0,1.654587e+09,131358000.0,1.218188e+09,NaN,NaN,210049000.0,3.255887e+09,1.575257e+09,16367000.0,NaN
2018,4.616277e+09,6.196830e+08,NaN,418893000.0,9865000.0,NaN,NaN,NaN,NaN,NaN,227701000.0,106838000.0,58160000.0,1.642315e+09,131271000.0,1.197640e+09,NaN,NaN,190550000.0,2.973962e+09,1.483047e+09,18465000.0,NaN
2019,4.919642e+09,6.709800e+08,NaN,447582000.0,20018000.0,36108000.0,NaN,NaN,NaN,NaN,386623000.0,248381000.0,54980000.0,1.790357e+09,131719000.0,1.193010e+09,NaN,NaN,180805000.0,3.129285e+09,1.377249e+09,25602000.0,NaN
2020,5.314677e+09,1.095274e+09,NaN,440199000.0,31378000.0,34990000.0,NaN,NaN,NaN,NaN,250277000.0,253132000.0,56185000.0,1.732470e+09,131050000.0,1.224886e+09,NaN,NaN,168496000.0,3.582207e+09,1.726906e+09,30330000.0,NaN
2021,4.749768e+09,6.584520e+08,NaN,410062000.0,47186000.0,32628000.0,NaN,NaN,NaN,NaN,270759000.0,195183000.0,57617000.0,1.804319e+09,130857000.0,1.196479e+09,NaN,NaN,152463000.0,2.945449e+09,1.302828e+09,33026000.0,NaN


In [153]:
kjmarshall.bs.df.loc[:,['Assets','AccountsPayable','AccruedIncomeTaxesCurrent','OperatingLeaseLiabilityCurrent','ContractWithCustomerLiability','ContractWithCustomerLiability','AccruedAdvertisingCurrent','DerivativeLiabilitiesCurrent','LiabilitiesOfDisposalGroupIncludingDiscontinuedOperationCurrent']]


tag,Assets,AccountsPayable,AccruedIncomeTaxesCurrent,OperatingLeaseLiabilityCurrent,ContractWithCustomerLiability,ContractWithCustomerLiability,AccruedAdvertisingCurrent,DerivativeLiabilitiesCurrent,LiabilitiesOfDisposalGroupIncludingDiscontinuedOperationCurrent
fy,,,,,,,,,
2014,4.968020e+09,611715000.0,37451000.0,NaN,NaN,NaN,NaN,NaN,NaN
2015,5.103850e+09,491378000.0,38747000.0,NaN,NaN,NaN,NaN,NaN,NaN
2016,4.742762e+09,412087000.0,15755000.0,NaN,NaN,NaN,NaN,NaN,NaN
2017,4.910474e+09,443113000.0,27862000.0,NaN,NaN,NaN,NaN,NaN,NaN
2018,4.616277e+09,418893000.0,9865000.0,NaN,NaN,NaN,NaN,NaN,NaN
2019,4.919642e+09,447582000.0,20018000.0,36108000.0,NaN,NaN,NaN,NaN,NaN
2020,5.314677e+09,440199000.0,31378000.0,34990000.0,NaN,NaN,NaN,NaN,NaN
2021,4.749768e+09,410062000.0,47186000.0,32628000.0,NaN,NaN,NaN,NaN,NaN


In [154]:
# kjmarshall.bs.df[['Assets','AccountsPayable','AccruedIncomeTaxesCurrent','OperatingLeaseLiabilityCurrent','ContractWithCustomerLiability','ContractWithCustomerLiability','AccruedAdvertisingCurrent','DerivativeLiabilitiesCurrent','LiabilitiesOfDisposalGroupIncludingDiscontinuedOperationCurrent']]
kjmarshall.bs.df['Assets'].sub(kjmarshall.bs.df['AccountsPayable']).sub(kjmarshall.bs.df['OperatingLeaseLiabilityCurrent'],fill_value=0)
# [['Assets','AccountsPayable','OperatingLeaseLiabilityCurrent']]
                  
                  

fy
2014    4.356305e+09
2015    4.612472e+09
2016    4.330675e+09
2017    4.467361e+09
2018    4.197384e+09
2019    4.435952e+09
2020    4.839488e+09
2021    4.307078e+09
dtype: float64

In [155]:
common_df = eu.create_common_metrics(mizrahi, safal, buffett, threebrians)

In [156]:
common_df

,Sales,Sales_YoY,NetReceivables as % of Sales,Sales-Mizrahi,SalesGrowth>ExpenseGrowth-3Brians,GrossProfit,GrossProfit_YoY,Gross Margin,Gross Margin_YoY,GrossMargin-Buffett,GrossProfit Growth-3Brians,CurrentRatio,CurrentRatio-Mizrahi,CurrentRatio-3Brians,Solvency (D/E Ratio),Solvency_YoY,Solvency (D/E)-Mizrahi,Solvency (D/E)-3Brians,NPM,NPM_YoY,NPM-Mizrahi,NPM Growth-3Brians,ROE,ROE_YoY,ROE-Mizrahi,OperatingMargin,OperatingMargin_YoY,OperatingMargin-Mizrahi,OperatingMargin Growth-3Brians,SGA,SGA%_YoY,SGA-Buffett,SGASlowdown?-3Brians,EPS-DILUTED,EPS_YoY,EPS-Mizrahi,EPS Growth-3Brians,FCF,FCF_YoY,FCF_Margin,FCF_Margin_YoY,FCF-Mizrahi,FCF Margin-Mizrahi,FCF-3Brians
fy,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,nan,nan,1.89837,Current Ratio : Good Level!,CurrentRatio: Robust,0.570415,NaN,nan,Solvency is Anti-fragile,NaN,NaN,nan,nan,NaN,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,nan,Not sure what to make of it,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,nan,nan,nan
2015,NaN,NaN,NaN,nan,nan,1477537000.0,NaN,NaN,NaN,nan,nan,1.935195,Current Ratio : Good Level!,CurrentRatio: Robust,0.942609,0.652498,Solvency (D/E) YoY above 10% => Not good,Solvency is Anti-fragile,NaN,NaN,nan,nan,0.155064,NaN,nan,NaN,NaN,nan,nan,0.658348,NaN,SGA b/w .3 & .8; Sometimes necessary to keep D...,Not sure what to make of it,1.93,NaN,nan,nan,258898000.0,NaN,NaN,NaN,nan,nan,nan
2016,3990487000.0,NaN,0.224221,nan,nan,1236798000.0,-0.162933,0.309937,NaN,Gross Margin implies tight competition,Gross Profit YoY flat/shrinking,1.978847,Current Ratio : Good Level!,CurrentRatio: Robust,0.901121,-0.044014,Solvency (D/E) YoY below 10% => Improving,Solvency is Anti-fragile,0.033193,NaN,nan,nan,0.080362,-0.481752,ROE YoY below 10%,0.069336,NaN,nan,nan,0.780545,0.185612,SGA b/w .3 & .8; Sometimes necessary to keep D...,SGA as % of gross profit going up :(,1.01,-0.476684,EPS YoY below 10%,EPS YoY is flat/shrinking,150777000.0,-0.41762,0.037784,NaN,FCF YoY below 10%,nan,FCF YoY going down :(
2017,3660831000.0,-0.08261,0.234021,Sales YoY < 10%,nan,1088953000.0,-0.119539,0.297461,-0.040253,Gross Margin implies tight competition,Gross Profit YoY flat/shrinking,2.058676,Current Ratio : Good Level!,CurrentRatio: Robust,0.906364,0.005818,Solvency (D/E) YoY below 10% => Improving,Solvency is Anti-fragile,0.000724,-0.978175,NPM YoY under 10%,NPM YoY is flat/shrinking,0.001603,-0.980055,ROE YoY below 10%,0.093185,0.343967,Operating Margin YoY up 10%,Oper Margin YoY is growing,0.828068,0.060885,SGA > .8; Too excessive :(,SGA as % of gross profit going up :(,0.02,-0.980198,EPS YoY below 10%,EPS YoY is flat/shrinking,249464000.0,0.654523,0.068144,0.803512,FCF YoY up 10%,FCF Margin YoY up 10%,FCF YoY going up :)
2018,3835699000.0,0.047767,0.206594,Sales YoY < 10%,nan,1190869000.0,0.093591,0.31047,0.043734,Gross Margin implies tight competition,Gross Profit YoY is growing,2.204625,Current Ratio : Good Level!,CurrentRatio: Robust,0.861485,-0.049516,Solvency (D/E) YoY below 10% => Improving,Solvency is Anti-fragile,0.027246,36.610679,NPM YoY up 10%,NPM YoY is growing,0.063635,38.701706,ROE YoY up 10%,0.059364,-0.36295,Operating Margin YoY below 10%,Oper Margin YoY is flat/shrinking,0.811663,-0.019811,SGA > .8; Too excessive :(,SGA as % of gross profit going down :),0.8,39.0,EPS YoY up 10%,EPS YoY is growing,106838000.0,-0.57173,0.027854,-0.591254,FCF YoY below 10%,FCF Margin YoY below 10%,FCF YoY going down :(
2019,3939697000.0,0.027113,0.201929,Sales YoY < 10%,nan,1289343000.0,0.082691,0.32727,0.054111,Gross Margin implies tight competition,Gross Profit YoY is growing,2.251233,Current Ratio : Good Level!,CurrentRatio: Robust,0.762963,-0.114362,Solvency (D/E) YoY below 10% => Improving,Solvency is Anti-fragile,0.060621,1.224935,NPM YoY up 10%,NPM YoY is growing,0.133397,1.096296,ROE YoY up 10%,0.098135,0.65312,Operating Margin YoY up 10%,Oper Margin YoY is growing,0.70827,-0.127384,SGA b/w .3 & .8; Sometimes necessary to keep D...,SGA as % of gross profit going down :),1.

In [312]:
eu.write_spreadsheet(ticker+'.xlsx', common_df, kjmarshall,mizrahi, safal, buffett, threebrians)